#  Capstone Project 2 - Loan Default

# Project Objective

The project is a binary classification problem which will attempt to create a model that can accurately predict if a borrower will default on their auto loan based on the demographic data, loan information and credit history provided?


### Data Wrangling

In this notebook we will load the raw data set and do peliminary exploration of the data, identify issues with data, modify data and data types and create new fields to aid in the next step which is EDA.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

from dateutil.relativedelta import relativedelta
from datetime import datetime

from library.sb_utils import save_file

#### Loading the data dictionary for reference

In [2]:
loan_desc = pd.read_csv('../data/data_dictionary.csv')
loan_desc = loan_desc.drop('Unnamed: 0', 1)
pd.set_option('display.max_colwidth', None)

loan_desc

Variable Name  \
0                              UniqueID   
1                          loan_default   
2                      disbursed_amount   
3                            asset_cost   
4                                   ltv   
5                             branch_id   
6                           supplier_id   
7                       manufacturer_id   
8                       Current_pincode   
9                         Date.of.Birth   
10                      Employment.Type   
11                        DisbursalDate   
12                             State_ID   
13                     Employee_code_ID   
14                    MobileNo_Avl_Flag   
15                          Aadhar_flag   
16                             PAN_flag   
17                         VoterID_flag   
18                         Driving_flag   
19                        Passport_flag   
20                    PERFORM_CNS.SCORE   
21        PERFORM_CNS.SCORE.DESCRIPTION   
22                      PRI.NO.OF.ACCTS   
23                     PRI.ACTIVE.ACCTS   
24                    PRI.OVERDUE.ACCTS   
25                  PRI.CURRENT.BALANCE   
26                PRI.SANCTIONED.AMOUNT   
27                 PRI.DISBURSED.AMOUNT   
28                      SEC.NO.OF.ACCTS   
29                     SEC.ACTIVE.ACCTS   
30                    SEC.OVERDUE.ACCTS   
31                  SEC.CURRENT.BALANCE   
32                SEC.SANCTIONED.AMOUNT   
33                 SEC.DISBURSED.AMOUNT   
34                   PRIMARY.INSTAL.AMT   
35                       SEC.INSTAL.AMT   
36         NEW.ACCTS.IN.LAST.SIX.MONTHS   
37  DELINQUENT.ACCTS.IN.LAST.SIX.MONTHS   
38                     AVERAGE.ACCT.AGE   
39                CREDIT.HISTORY.LENGTH   
40                      NO.OF_INQUIRIES   

                                                                           Description  \
0                                                             Identifier for customers   
1                                         Payment default in the first EMI on due date   
2                                                             Amount of Loan disbursed   
3                                                                    Cost of the Asset   
4                                                           Loan to Value of the asset   
5                                                  Branch where the loan was disbursed   
6                                          Vehicle Dealer where the loan was disbursed   
7                                          Vehicle manufacturer(Hero, Honda, TVS etc.)   
8                                                      Current pincode of the customer   
9                                                        Date of birth of the customer   
10                            Employment Type of the customer (Salaried/Self Employed)   
11                                                                Date of disbursement   
12                                                               State of disbursement   
13                            Employee of the organization who logged the disbursement   
14                          if Mobile no. was shared by the customer then flagged as 1   
15                              if aadhar was shared by the customer then flagged as 1   
16                                 if pan was shared by the customer then flagged as 1   
17                              if voter  was shared by the customer then flagged as 1   
18                                  if DL was shared by the customer then flagged as 1   
19                            if passport was shared by the customer then flagged as 1   
20                                                                        Bureau Score   
21                                                            Bureau score description   
22              count of total loans taken by the customer at the time of disbursement   
23             count of active loans taken by the customer 

#### dataset load

Dataset was downloaded from https://www.kaggle.com/mamtadhaker/lt-vehicle-loan-default-prediction?select=train.csv 


In [3]:
loan_data = pd.read_csv('../data/train.csv')

In [4]:
loan_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 233154 entries, 0 to 233153
Data columns (total 41 columns):
 #   Column                               Non-Null Count   Dtype  
---  ------                               --------------   -----  
 0   UniqueID                             233154 non-null  int64  
 1   disbursed_amount                     233154 non-null  int64  
 2   asset_cost                           233154 non-null  int64  
 3   ltv                                  233154 non-null  float64
 4   branch_id                            233154 non-null  int64  
 5   supplier_id                          233154 non-null  int64  
 6   manufacturer_id                      233154 non-null  int64  
 7   Current_pincode_ID                   233154 non-null  int64  
 8   Date.of.Birth                        233154 non-null  object 
 9   Employment.Type                      225493 non-null  object 
 10  DisbursalDate                        233154 non-null  object 
 11  State_ID     

replacing dots with underscores in the column names to make it more readable.

In [5]:
loan_data.columns = loan_data.columns.str.replace('.','_')

<ipython-input-5-db274bf2b123>:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  loan_data.columns = loan_data.columns.str.replace('.','_')


In [6]:
loan_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 233154 entries, 0 to 233153
Data columns (total 41 columns):
 #   Column                               Non-Null Count   Dtype  
---  ------                               --------------   -----  
 0   UniqueID                             233154 non-null  int64  
 1   disbursed_amount                     233154 non-null  int64  
 2   asset_cost                           233154 non-null  int64  
 3   ltv                                  233154 non-null  float64
 4   branch_id                            233154 non-null  int64  
 5   supplier_id                          233154 non-null  int64  
 6   manufacturer_id                      233154 non-null  int64  
 7   Current_pincode_ID                   233154 non-null  int64  
 8   Date_of_Birth                        233154 non-null  object 
 9   Employment_Type                      225493 non-null  object 
 10  DisbursalDate                        233154 non-null  object 
 11  State_ID     

In [7]:
pd.set_option('display.max_columns', None)
loan_data.head(10)

UniqueID  disbursed_amount  asset_cost    ltv  branch_id  supplier_id  \
0    420825             50578       58400  89.55         67        22807   
1    537409             47145       65550  73.23         67        22807   
2    417566             53278       61360  89.63         67        22807   
3    624493             57513       66113  88.48         67        22807   
4    539055             52378       60300  88.39         67        22807   
5    518279             54513       61900  89.66         67        22807   
6    529269             46349       61500  76.42         67        22807   
7    510278             43894       61900  71.89         67        22807   
8    490213             53713       61973  89.56         67        22807   
9    510980             52603       61300  86.95         67        22807   

   manufacturer_id  Current_pincode_ID Date_of_Birth Employment_Type  \
0               45                1441      01-01-84        Salaried   
1               45                1502      31-07-85   Self employed   
2               45                1497      24-08-85   Self employed   
3               45                1501      30-12-93   Self employed   
4               45                1495      09-12-77   Self employed   
5               45                1501      08-09-90   Self employed   
6               45                1502      01-06-88        Salaried   
7               45                1501      04-10-89        Salaried   
8               45                1497      15-11-91   Self employed   
9               45                1492      01-06-68        Salaried   

  DisbursalDate  State_ID  Employee_code_ID  MobileNo_Avl_Flag  Aadhar_flag  \
0      03-08-18         6              1998                  1            1   
1      26-09-18         6              1998                  1            1   
2      01-08-18         6              1998                  1            1   
3      26-10-18         6              1998                  1            1   
4      26-09-18         6              1998                  1            1   
5      19-09-18         6              1998                  1            1   
6      23-09-18         6              1998                  1            1   
7      16-09-18         6              1998                  1            1   
8      05-09-18         6              1998                  1            1   
9      16-09-18         6              1998                  1            0   

   PAN_flag  VoterID_flag  Driving_flag  Passport_flag  PERFORM_CNS_SCORE  \
0         0             0             0              0                  0   
1         0             0             0              0                598   
2         0             0             0              0                  0   
3         0             0             0              0                305   
4         0             0             0              0                  0   
5         0             0             0              0                825   
6         0             0             0              0                  0   
7         0             0             0              0                 17   
8         0             0             0              0                718   
9         0             1             0              0                818   

                           PERFORM_CNS_SCORE_DESCRIPTION  PRI_NO_OF_ACCTS  \
0                            No Bureau History Available                0   
1                                          I-Medium Risk                1   
2                            No Bureau History Available                0   
3                                       L-Very High Risk                3   
4                            No Bureau History Available                0   
5                                        A-Very Low Risk                2   
6                            No Bureau History Available                0   
7  Not Scored: Not Enough Info availabl

In [8]:
loan_data.describe().T

count           mean           std  \
UniqueID                             233154.0  535917.573376  6.831569e+04   
disbursed_amount                     233154.0   54356.993528  1.297131e+04   
asset_cost                           233154.0   75865.068144  1.894478e+04   
ltv                                  233154.0      74.746530  1.145664e+01   
branch_id                            233154.0      72.936094  6.983499e+01   
supplier_id                          233154.0   19638.635035  3.491950e+03   
manufacturer_id                      233154.0      69.028054  2.214130e+01   
Current_pincode_ID                   233154.0    3396.880247  2.238148e+03   
State_ID                             233154.0       7.262243  4.482230e+00   
Employee_code_ID                     233154.0    1549.477148  9.752613e+02   
MobileNo_Avl_Flag                    233154.0       1.000000  0.000000e+00   
Aadhar_flag                          233154.0       0.840320  3.663097e-01   
PAN_flag                             233154.0       0.075577  2.643201e-01   
VoterID_flag                         233154.0       0.144943  3.520439e-01   
Driving_flag                         233154.0       0.023242  1.506720e-01   
Passport_flag                        233154.0       0.002127  4.607421e-02   
PERFORM_CNS_SCORE                    233154.0     289.462994  3.383748e+02   
PRI_NO_OF_ACCTS                      233154.0       2.440636  5.217233e+00   
PRI_ACTIVE_ACCTS                     233154.0       1.039896  1.941496e+00   
PRI_OVERDUE_ACCTS                    233154.0       0.156549  5.487867e-01   
PRI_CURRENT_BALANCE                  233154.0  165900.076936  9.422736e+05   
PRI_SANCTIONED_AMOUNT                233154.0  218503.855323  2.374794e+06   
PRI_DISBURSED_AMOUNT                 233154.0  218065.898655  2.377744e+06   
SEC_NO_OF_ACCTS                      233154.0       0.059081  6.267946e-01   
SEC_ACTIVE_ACCTS                     233154.0       0.027703  3.160566e-01   
SEC_OVERDUE_ACCTS                    233154.0       0.007244  1.110789e-01   
SEC_CURRENT_BALANCE                  233154.0    5427.792819  1.702370e+05   
SEC_SANCTIONED_AMOUNT                233154.0    7295.923347  1.831560e+05   
SEC_DISBURSED_AMOUNT                 233154.0    7179.997873  1.825925e+05   
PRIMARY_INSTAL_AMT                   233154.0   13105.481720  1.513679e+05   
SEC_INSTAL_AMT                       233154.0     323.268449  1.555369e+04   
NEW_ACCTS_IN_LAST_SIX_MONTHS         233154.0       0.381833  9.551067e-01   
DELINQUENT_ACCTS_IN_LAST_SIX_MONTHS  233154.0       0.097481  3.844390e-01   
NO_OF_INQUIRIES                      233154.0       0.206615  7.064977e-01   
loan_default                         233154.0       0.217071  4.122523e-01   

                                            min        25%       50%  \
UniqueID                              417428.00  476786.25  535978.5   
disbursed_amount                       13320.00   47145.00   53803.0   
asset_cost                             37000.00   65717.00   70946.0   
ltv                                       10.03      68.88      76.8   
branch_id                                  1.00      14.00      61.0   
supplier_id                            10524.00   16535.00   20333.0   
manufacturer_id                           45.00      48.00      86.0   
Current_pincode_ID                         1.00    1511.00    2970.0   
State_ID                                   1.00       4.00       6.0   
Employee_code_ID                           1.00     713.00    1451.0   
MobileNo_Avl_Flag                          1.00       1.00       1.0   
Aadhar_flag                                0.00       1.00       1.0   
PAN_flag                                   0.00       0.00       0.0   
VoterID_flag                               0.00       0.00       0.0   
Driving_flag                               0.00       0.00       0.0   
Passport_flag                              0.00       0.00       0.0   
PERFORM

the columns Date_of_Brith and DisbursalDate formats need to be converted from objects to date

In [9]:
#loan_data['Date_of_Birth'] = datetime.strptime(loan_data['Date_of_Birth'], '%d/%m/%y')
#loan_data['Date_of_Birth'] = pd.to_datetime(loan_data['Date_of_Birth'], format='%d%m%y') changed to below format needed to be format='%d-%m-%y'
loan_data['Date_of_Birth_date'] = pd.to_datetime(loan_data['Date_of_Birth'], infer_datetime_format=True)
loan_data['Date_of_Birth_date']

0        1984-01-01
1        1985-07-31
2        1985-08-24
3        1993-12-30
4        1977-09-12
            ...    
233149   1988-01-08
233150   1988-05-12
233151   1976-01-06
233152   1994-03-26
233153   1984-02-18
Name: Date_of_Birth_date, Length: 233154, dtype: datetime64[ns]

In [10]:
loan_data['DisbursalDate_date'] = pd.to_datetime(loan_data['DisbursalDate'], infer_datetime_format=True)
loan_data['DisbursalDate_date']

0        2018-03-08
1        2018-09-26
2        2018-01-08
3        2018-10-26
4        2018-09-26
            ...    
233149   2018-10-26
233150   2018-10-23
233151   2018-10-24
233152   2018-09-29
233153   2018-10-27
Name: DisbursalDate_date, Length: 233154, dtype: datetime64[ns]

Birthday on its own is not very useful, creating an AGE column at time of loan using Date_of_Birth and DisbursalDate

In [11]:
#from dateutil.relativedelta import relativedelta

#difference_in_years = relativedelta(end_date, start_date).years
#loan_data['age_of_cust'] = relativedelta(loan_data['DisbursalDate'], loan_data['Date_of_Birth']).years
#recieved value error, using a lambda expression to resolve.
loan_data['age_of_cust'] = loan_data.apply(lambda x: relativedelta(x['DisbursalDate_date'], x['Date_of_Birth_date']).years, axis=1)


In [12]:
loan_data['age_of_cust'].describe()

count    233154.000000
mean         22.406783
std          26.028039
min         -52.000000
25%          23.000000
50%          29.000000
75%          36.000000
max          47.000000
Name: age_of_cust, dtype: float64

In [13]:
loan_data[loan_data['age_of_cust'] == -52]

UniqueID  disbursed_amount  asset_cost    ltv  branch_id  supplier_id  \
161       484275             54305       64760  85.00         34        15196   
219       551505             51103       71524  72.42         34        15196   
222       530577             53403       69542  77.79         34        15196   
484       600315             48349       66283  73.93         34        15142   
528       596385             56481       67320  85.00         34        15142   
...          ...               ...         ...    ...        ...          ...   
230846    592352             42050       65400  66.67         78        18396   
231159    423660             58013       69020  86.21         74        21347   
231442    594070             58159       74400  79.84        135        23003   
232094    436473             58213       66800  89.07        135        23618   
232829    537585             54959       72832  76.89        138        15996   

        manufacturer_id  Current_pincode_ID Date_of_Birth Employment_Type  \
161                  86                1050      09-06-70   Self employed   
219                  86                 992      23-10-70        Salaried   
222                  86                1051      20-11-70   Self employed   
484                  86                1045      11-06-70        Salaried   
528                  86                 984      23-11-70   Self employed   
...                 ...                 ...           ...             ...   
230846               45                2065      15-11-70   Self employed   
231159               86                2578      09-09-70        Salaried   
231442               49                1700      15-11-70        Salaried   
232094               45                2376      22-08-70   Self employed   
232829              120                3359      10-06-70        Salaried   

       DisbursalDate  State_ID  Employee_code_ID  MobileNo_Avl_Flag  \
161         31-08-18         6              1863                  1   
219         30-09-18         6              1863                  1   
222         23-09-18         6              1863                  1   
484         22-10-18         6               864                  1   
528         21-10-18         6               864                  1   
...              ...       ...               ...                ...   
230846      20-10-18         4              2230                  1   
231159      06-08-18         4               176                  1   
231442      21-10-18         4              1823                  1   
232094      13-08-18         4              2044                  1   
232829      26-09-18         9              2690                  1   

        Aadhar_flag  PAN_flag  VoterID_flag  Driving_flag  Passport_flag  \
161               1         0             0             0              0   
219               0         0             0             1              0   
222               1         0             0             0              0   
484               1         0             0             0              0   
528               1         0             0             0              0   
...             ...       ...           ...           ...            ...   
230846            1         0             0             0              0   
231159            1         0             0             0              0   
231442            1         0             0             0              0   
232094            1         0             0             0              0   
232829            1         0             0             0              0   

        PERFORM_CNS_SCORE PERFORM_CNS_SCORE_DESCRIPTION  PRI_NO_OF_ACCTS  \
161                     0   No Bureau History Available                0   
219                   836               A-Very Low Risk                1   
222                   746               C-Very Low Risk                2   
484                   705               

Looks like an issue with the date convert 15-11-70 was converted to 2070-11-15 which is causing us to calculate negative ages, will subtract 100 years from date of birth where birth date greater than 2018-01-01

In [14]:
now = pd.Timestamp('2018-01-01')
loan_data['Date_of_Birth_date'] = loan_data['Date_of_Birth_date'].where(loan_data['Date_of_Birth_date'] < now, loan_data['Date_of_Birth_date'] -  np.timedelta64(100, 'Y'))
    
print(f'min date: ',loan_data['Date_of_Birth_date'].min())
print(f'max date: ',loan_data['Date_of_Birth_date'].max())
#loan_data['Date_of_Birth_date'] = pd.to_datetime(loan_data['Date_of_Birth'], infer_datetime_format=True)
#loan_data['Date_of_Birth_date']

min date:  1949-09-15 18:00:00
max date:  2000-12-08 00:00:00


In [15]:
loan_data['age_of_cust'] = loan_data.apply(lambda x: relativedelta(x['DisbursalDate_date'], x['Date_of_Birth_date']).years, axis=1)

In [16]:
loan_data['age_of_cust'].describe()

count    233154.000000
mean         33.903579
std           9.828921
min          17.000000
25%          26.000000
50%          32.000000
75%          41.000000
max          69.000000
Name: age_of_cust, dtype: float64

AVERAGE_ACCT_AGE and CREDIT_HISTORY_LENGTH are in this format 0yrs 0mon need to convert to numeric months, using a function 

In [17]:

def to_month(x):
    parsed = x.split(' ')
    year = int(parsed[0].replace('yrs',''))
    month = int(parsed[1].replace('mon',''))
    return (year*12)+month

In [18]:
loan_data['AVERAGE_ACCT_AGE_MONTHS'] = loan_data['AVERAGE_ACCT_AGE'].apply(to_month)
loan_data['AVERAGE_ACCT_AGE_MONTHS'].value_counts()

0      119373
6        6028
7        5366
11       5237
10       5143
        ...  
167         1
292         1
227         1
158         1
192         1
Name: AVERAGE_ACCT_AGE_MONTHS, Length: 192, dtype: int64

In [19]:
loan_data['CREDIT_HISTORY_LENGTH_MONTHS'] = loan_data['CREDIT_HISTORY_LENGTH'].apply(to_month)
loan_data['CREDIT_HISTORY_LENGTH_MONTHS'].value_counts()

0      119127
6        4761
25       4745
7        4017
24       3833
        ...  
279         1
240         1
369         1
277         1
220         1
Name: CREDIT_HISTORY_LENGTH_MONTHS, Length: 294, dtype: int64

In [20]:
missing = pd.concat([loan_data.isnull().sum(), 100 * loan_data.isnull().mean()], axis=1)
missing.columns=['count', '%']
missing.sort_values(by=['count','%'],ascending=False)

count         %
Employment_Type                       7661  3.285811
UniqueID                                 0  0.000000
disbursed_amount                         0  0.000000
asset_cost                               0  0.000000
ltv                                      0  0.000000
branch_id                                0  0.000000
supplier_id                              0  0.000000
manufacturer_id                          0  0.000000
Current_pincode_ID                       0  0.000000
Date_of_Birth                            0  0.000000
DisbursalDate                            0  0.000000
State_ID                                 0  0.000000
Employee_code_ID                         0  0.000000
MobileNo_Avl_Flag                        0  0.000000
Aadhar_flag                              0  0.000000
PAN_flag                                 0  0.000000
VoterID_flag                             0  0.000000
Driving_flag                             0  0.000000
Passport_flag                            0  0.000000
PERFORM_CNS_SCORE                        0  0.000000
PERFORM_CNS_SCORE_DESCRIPTION            0  0.000000
PRI_NO_OF_ACCTS                          0  0.000000
PRI_ACTIVE_ACCTS                         0  0.000000
PRI_OVERDUE_ACCTS                        0  0.000000
PRI_CURRENT_BALANCE                      0  0.000000
PRI_SANCTIONED_AMOUNT                    0  0.000000
PRI_DISBURSED_AMOUNT                     0  0.000000
SEC_NO_OF_ACCTS                          0  0.000000
SEC_ACTIVE_ACCTS                         0  0.000000
SEC_OVERDUE_ACCTS                        0  0.000000
SEC_CURRENT_BALANCE                      0  0.000000
SEC_SANCTIONED_AMOUNT                    0  0.000000
SEC_DISBURSED_AMOUNT                     0  0.000000
PRIMARY_INSTAL_AMT                       0  0.000000
SEC_INSTAL_AMT                           0  0.000000
NEW_ACCTS_IN_LAST_SIX_MONTHS             0  0.000000
DELINQUENT_ACCTS_IN_LAST_SIX_MONTHS      0  0.000000
AVERAGE_ACCT_AGE                         0  0.000000
CREDIT_HISTORY_LENGTH                    0  0.000000
NO_OF_INQUIRIES                          0  0.000000
loan_default                             0  0.000000
Date_of_Birth_date                       0  0.000000
DisbursalDate_date                       0  0.000000
age_of_cust                              0  0.000000
AVERAGE_ACCT_AGE_MONTHS                  0  0.000000
CREDIT_HISTORY_LENGTH_MONTHS             0  0.000000

In [21]:
missing = pd.concat([loan_data.isna().sum(), 100 * loan_data.isna().mean()], axis=1)
missing.columns=['count', '%']
missing.sort_values(by=['count','%'],ascending=False)

count         %
Employment_Type                       7661  3.285811
UniqueID                                 0  0.000000
disbursed_amount                         0  0.000000
asset_cost                               0  0.000000
ltv                                      0  0.000000
branch_id                                0  0.000000
supplier_id                              0  0.000000
manufacturer_id                          0  0.000000
Current_pincode_ID                       0  0.000000
Date_of_Birth                            0  0.000000
DisbursalDate                            0  0.000000
State_ID                                 0  0.000000
Employee_code_ID                         0  0.000000
MobileNo_Avl_Flag                        0  0.000000
Aadhar_flag                              0  0.000000
PAN_flag                                 0  0.000000
VoterID_flag                             0  0.000000
Driving_flag                             0  0.000000
Passport_flag                            0  0.000000
PERFORM_CNS_SCORE                        0  0.000000
PERFORM_CNS_SCORE_DESCRIPTION            0  0.000000
PRI_NO_OF_ACCTS                          0  0.000000
PRI_ACTIVE_ACCTS                         0  0.000000
PRI_OVERDUE_ACCTS                        0  0.000000
PRI_CURRENT_BALANCE                      0  0.000000
PRI_SANCTIONED_AMOUNT                    0  0.000000
PRI_DISBURSED_AMOUNT                     0  0.000000
SEC_NO_OF_ACCTS                          0  0.000000
SEC_ACTIVE_ACCTS                         0  0.000000
SEC_OVERDUE_ACCTS                        0  0.000000
SEC_CURRENT_BALANCE                      0  0.000000
SEC_SANCTIONED_AMOUNT                    0  0.000000
SEC_DISBURSED_AMOUNT                     0  0.000000
PRIMARY_INSTAL_AMT                       0  0.000000
SEC_INSTAL_AMT                           0  0.000000
NEW_ACCTS_IN_LAST_SIX_MONTHS             0  0.000000
DELINQUENT_ACCTS_IN_LAST_SIX_MONTHS      0  0.000000
AVERAGE_ACCT_AGE                         0  0.000000
CREDIT_HISTORY_LENGTH                    0  0.000000
NO_OF_INQUIRIES                          0  0.000000
loan_default                             0  0.000000
Date_of_Birth_date                       0  0.000000
DisbursalDate_date                       0  0.000000
age_of_cust                              0  0.000000
AVERAGE_ACCT_AGE_MONTHS                  0  0.000000
CREDIT_HISTORY_LENGTH_MONTHS             0  0.000000

In [22]:
loan_data['Employment_Type'].value_counts()

Self employed    127635
Salaried          97858
Name: Employment_Type, dtype: int64

### 7661 blank Employment types - we will address that in next notebook

In [23]:
loan_data['loan_default'].value_counts()

0    182543
1     50611
Name: loan_default, dtype: int64

Loan_default is what we are trying to predict so we have an imbalanced classification.

In [24]:
loan_data.describe().T

count           mean           std  \
UniqueID                             233154.0  535917.573376  6.831569e+04   
disbursed_amount                     233154.0   54356.993528  1.297131e+04   
asset_cost                           233154.0   75865.068144  1.894478e+04   
ltv                                  233154.0      74.746530  1.145664e+01   
branch_id                            233154.0      72.936094  6.983499e+01   
supplier_id                          233154.0   19638.635035  3.491950e+03   
manufacturer_id                      233154.0      69.028054  2.214130e+01   
Current_pincode_ID                   233154.0    3396.880247  2.238148e+03   
State_ID                             233154.0       7.262243  4.482230e+00   
Employee_code_ID                     233154.0    1549.477148  9.752613e+02   
MobileNo_Avl_Flag                    233154.0       1.000000  0.000000e+00   
Aadhar_flag                          233154.0       0.840320  3.663097e-01   
PAN_flag                             233154.0       0.075577  2.643201e-01   
VoterID_flag                         233154.0       0.144943  3.520439e-01   
Driving_flag                         233154.0       0.023242  1.506720e-01   
Passport_flag                        233154.0       0.002127  4.607421e-02   
PERFORM_CNS_SCORE                    233154.0     289.462994  3.383748e+02   
PRI_NO_OF_ACCTS                      233154.0       2.440636  5.217233e+00   
PRI_ACTIVE_ACCTS                     233154.0       1.039896  1.941496e+00   
PRI_OVERDUE_ACCTS                    233154.0       0.156549  5.487867e-01   
PRI_CURRENT_BALANCE                  233154.0  165900.076936  9.422736e+05   
PRI_SANCTIONED_AMOUNT                233154.0  218503.855323  2.374794e+06   
PRI_DISBURSED_AMOUNT                 233154.0  218065.898655  2.377744e+06   
SEC_NO_OF_ACCTS                      233154.0       0.059081  6.267946e-01   
SEC_ACTIVE_ACCTS                     233154.0       0.027703  3.160566e-01   
SEC_OVERDUE_ACCTS                    233154.0       0.007244  1.110789e-01   
SEC_CURRENT_BALANCE                  233154.0    5427.792819  1.702370e+05   
SEC_SANCTIONED_AMOUNT                233154.0    7295.923347  1.831560e+05   
SEC_DISBURSED_AMOUNT                 233154.0    7179.997873  1.825925e+05   
PRIMARY_INSTAL_AMT                   233154.0   13105.481720  1.513679e+05   
SEC_INSTAL_AMT                       233154.0     323.268449  1.555369e+04   
NEW_ACCTS_IN_LAST_SIX_MONTHS         233154.0       0.381833  9.551067e-01   
DELINQUENT_ACCTS_IN_LAST_SIX_MONTHS  233154.0       0.097481  3.844390e-01   
NO_OF_INQUIRIES                      233154.0       0.206615  7.064977e-01   
loan_default                         233154.0       0.217071  4.122523e-01   
age_of_cust                          233154.0      33.903579  9.828921e+00   
AVERAGE_ACCT_AGE_MONTHS              233154.0       8.915764  1.510642e+01   
CREDIT_HISTORY_LENGTH_MONTHS         233154.0      16.252404  2.858125e+01   

                                            min        25%       50%  \
UniqueID                              417428.00  476786.25  535978.5   
disbursed_amount                       13320.00   47145.00   53803.0   
asset_cost                             37000.00   65717.00   70946.0   
ltv                                       10.03      68.88      76.8   
branch_id                                  1.00      14.00      61.0   
supplier_id                            10524.00   16535.00   20333.0   
manufacturer_id                           45.00      48.00      86.0   
Current_pincode_ID                         1.00    1511.00    2970.0   
State_ID                                   1.00       4.00       6.0   
Employee_code_ID                           1.00     713.00    1451.0   
MobileNo_Avl_Flag                          1.00       1.00       1.0   
Aadhar_flag                                0.00       1.00       1.0   
PAN_flag                                   0.00       0.00   

In [25]:
loan_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 233154 entries, 0 to 233153
Data columns (total 46 columns):
 #   Column                               Non-Null Count   Dtype         
---  ------                               --------------   -----         
 0   UniqueID                             233154 non-null  int64         
 1   disbursed_amount                     233154 non-null  int64         
 2   asset_cost                           233154 non-null  int64         
 3   ltv                                  233154 non-null  float64       
 4   branch_id                            233154 non-null  int64         
 5   supplier_id                          233154 non-null  int64         
 6   manufacturer_id                      233154 non-null  int64         
 7   Current_pincode_ID                   233154 non-null  int64         
 8   Date_of_Birth                        233154 non-null  object        
 9   Employment_Type                      225493 non-null  object        
 

writing out the modified file

In [26]:
datapath = '../data'
save_file(loan_data, 'loan_data.csv', datapath)

Writing file.  "../data\loan_data.csv"


### Summary

This dataset was already pretty clean,  in this notebook AVERAGE_ACCT_AGE and CREDIT_HISTORY_LENGTH text fields were converted to numeric and Date_of_Birth and DisbursalDate were converted to date format.  A new field of age of customer was engineered from the date of birth and disbursal date.  We do have some outliers and missing data that was identified in this notebook but will be addressed in the next notebook(Exploratory Data Anaylsis).